In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
df_train = pd.read_csv('C:/Users/evely/OneDrive/Desktop/CS 4661/healthcare/train_data.csv')
#train_dict = pd.read_csv('C:/Users/evely/OneDrive/Desktop/CS 4661/healthcare/train_data_dictionary.csv')
df_test = pd.read_csv('C:/Users/evely/OneDrive/Desktop/CS 4661/healthcare/test_data.csv')
#samp = pd.read_csv('C:/Users/evely/OneDrive/Desktop/CS 4661/healthcare/sample_sub.csv')

In [3]:
df_train.tail(20)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
318418,318419,17,e,1,X,2,gynecology,R,E,4.0,38816,32.0,Urgent,Minor,2,31-40,4499.0,0-10
318419,318420,16,c,3,Z,1,gynecology,S,A,4.0,92559,6.0,Trauma,Moderate,2,41-50,4339.0,31-40
318420,318421,3,c,3,Z,3,anesthesia,Q,A,3.0,79339,20.0,Urgent,Moderate,2,71-80,4667.0,11-20
318421,318422,19,a,7,Y,2,gynecology,S,C,2.0,70282,8.0,Emergency,Moderate,2,41-50,4277.0,0-10
318422,318423,22,g,9,Y,3,anesthesia,R,B,2.0,87939,23.0,Emergency,Minor,3,41-50,4510.0,0-10
318423,318424,23,a,6,X,2,anesthesia,Q,F,4.0,131049,1.0,Urgent,Moderate,2,31-40,3190.0,11-20
318424,318425,6,a,6,X,2,radiotherapy,R,F,4.0,22709,2.0,Urgent,Moderate,2,51-60,4455.0,11-20
318425,318426,15,c,5,Z,2,anesthesia,S,F,4.0,50508,12.0,Emergency,Minor,2,11-20,8052.0,11-20
318426,318427,17,e,1,X,3,gynecology,R,E,4.0,109647,32.0,Emergency,Minor,3,0-10,4185.0,0-10
318427,318428,20,b,2,Y,2,TB & Chest disease,S,D,3.0,79108,5.0,Urgent,Minor,4,51-60,4271.0,31-40


df = pd.get_dummies(df_train,prefix=['Department'], columns = ['Department'], drop_first=True)

df

In [4]:
def normalize(df):
  #dept = df['Department'].unique()
  #new_dept = dict(zip(dept, range(len(dept)))) 
  #df['Department'].replace(new_dept, inplace=True)

  hrc = df['Hospital_region_code'].unique()
  new_hrc = dict(zip(hrc, range(len(hrc))))
  df['Hospital_region_code'].replace(new_hrc, inplace=True)

  ward = df['Ward_Type'].unique()
  new_ward = dict(zip(ward, range(len(ward))))
  df['Ward_Type'].replace(new_ward, inplace=True)

  wfc = df['Ward_Facility_Code'].unique()
  new_wfc = dict(zip(wfc, range(len(wfc))))
  df['Ward_Facility_Code'].replace(new_wfc, inplace=True)

  #toa = df['Type of Admission'].unique()
  #new_toa = dict(zip(toa, range(len(toa))))
  #df['Type of Admission'].replace(new_toa, inplace=True)

  soi = df['Severity of Illness'].unique()
  new_soi = dict(zip(soi, range(len(soi))))
  df['Severity of Illness'].replace(new_soi, inplace=True)

  age = df['Age'].unique() 
  new_age = dict(zip(age, range(len(age))))
  df['Age'].replace(new_age, inplace=True)

  htc = df['Hospital_type_code'].unique()
  new_htc = dict(zip(htc, range(len(htc))))
  df['Hospital_type_code'].replace(new_htc, inplace=True)

In [5]:
#OneHotEncoder after normalization
df_train = pd.concat([df_train,pd.get_dummies(df_train['Department'], 
            prefix='Department')], axis=1).drop(['Department'],axis=1)
df_train.reset_index(drop=True, inplace=True)

df_train = pd.concat([df_train,pd.get_dummies(df_train['Type of Admission'], 
                prefix='Admission')], axis=1).drop(['Type of Admission'],axis=1)
df_train.reset_index(drop=True, inplace=True)

In [6]:
normalize(df_train)
normalize(df_test)
df_train.head(40)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,...,Admission_Deposit,Stay,Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,Admission_Emergency,Admission_Trauma,Admission_Urgent
0,1,8,0,3,0,3,0,0,2.0,31397,...,4911.0,0-10,0,0,0,1,0,1,0,0
1,2,2,0,5,0,2,1,0,2.0,31397,...,5954.0,41-50,0,0,0,1,0,0,1,0
2,3,10,1,1,1,2,1,1,2.0,31397,...,4745.0,31-40,0,1,0,0,0,0,1,0
3,4,26,2,2,2,2,0,2,2.0,31397,...,7272.0,41-50,0,0,0,1,0,0,1,0
4,5,26,2,2,2,2,1,2,2.0,31397,...,5558.0,41-50,0,0,0,1,0,0,1,0
5,6,23,3,6,1,2,1,0,2.0,31397,...,4449.0,11-20,0,1,0,0,0,0,1,0
6,7,32,4,9,2,1,1,3,3.0,31397,...,6167.0,0-10,0,0,0,1,0,1,0,0
7,8,23,3,6,1,4,2,0,3.0,31397,...,5571.0,41-50,0,0,0,1,0,0,1,0
8,9,1,5,10,2,2,0,3,4.0,31397,...,7223.0,51-60,0,0,1,0,0,0,1,0
9,10,10,1,1,1,2,1,1,3.0,31397,...,6056.0,31-40,0,0,1,0,0,0,1,0


features = ["case_id", "Hospital_code", "Available Extra Rooms in Hospital", "Age","Department_anesthesia","Department_gynecology","Department_radiotherapy","Department_surgery"]

In [7]:
features = ["case_id", "Hospital_code", "Available Extra Rooms in Hospital", "Age","Department_anesthesia","Department_gynecology","Department_radiotherapy","Department_surgery","Admission_Emergency","Admission_Trauma","Admission_Urgent"]
X = df_train[features]
y = df_train['Stay']
X

,case_id,Hospital_code,Available Extra Rooms in Hospital,Age,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,Admission_Emergency,Admission_Trauma,Admission_Urgent
0,1,8,3,0,0,0,1,0,1,0,0
1,2,2,2,0,0,0,1,0,0,1,0
2,3,10,2,0,1,0,0,0,0,1,0
3,4,26,2,0,0,0,1,0,0,1,0
4,5,26,2,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
318433,318434,6,3,3,0,0,1,0,1,0,0
318434,318435,24,2,4,1,0,0,0,0,0,1
318435,318436,7,3,1,0,1,0,0,1,0,0
318436,318437,11,3,7,1,0,0,0,0,1,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=7505, random_state=500)

k = 5000
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)
y_predict = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_predict)

print(accuracy)

0.2822118587608261


In [9]:
results = pd.DataFrame()

results['Actual'] = y_test 
results['Prediction'] = y_predict 

print(results)

                    Actual Prediction
85561                21-30      21-30
198007               31-40      21-30
222018  More than 100 Days      21-30
44462                11-20      21-30
122285               21-30      21-30
...                    ...        ...
63465                11-20      21-30
33615                71-80      21-30
85490                11-20      21-30
21731                11-20      21-30
315990               11-20      11-20

[7505 rows x 2 columns]


# Descision Tree

In [10]:
my_decisiontree = DecisionTreeClassifier()

In [11]:
my_decisiontree.fit(X_train, y_train)

DecisionTreeClassifier()

In [12]:
y_predict = my_decisiontree.predict(X_test)

print(y_predict)

['21-30' '21-30' '31-40' ... '11-20' '31-40' '21-30']


In [13]:
accuracy = accuracy_score(y_test, y_predict)

print(accuracy)

0.23197868087941373


----------------------------------------------------------------------------------------

In [14]:
features = ["Department_anesthesia","Department_gynecology","Department_radiotherapy","Department_surgery"]
X = df_train[features]
y = df_train['Stay']
X

,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
318433,0,0,1,0
318434,1,0,0,0
318435,0,1,0,0
318436,1,0,0,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=7505, random_state=500)

k = 5000
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)
y_predict = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_predict)

print(accuracy)

In [ ]:
results = pd.DataFrame()

results['Actual'] = y_test 
results['Prediction'] = y_predict 

print(results)

In [ ]:
my_decisiontree.fit(X_train, y_train)
y_predict = my_decisiontree2.predict(X_test)

print(y_predict)

accuracy = accuracy_score(y_test, y_predict)

print('\n',"Accuracy: ", accuracy) 

In [ ]:
features = ["Admission_Emergency","Admission_Trauma","Admission_Urgent"]
X = df_train[features]
y = df_train['Stay']
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=7505, random_state=500)

k = 5000
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)
y_predict = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_predict)

print(accuracy)

In [ ]:
results = pd.DataFrame()

results['Actual'] = y_test 
results['Prediction'] = y_predict 

print(results)

In [ ]:
my_decisiontree.fit(X_train, y_train)
y_predict = my_decisiontree3.predict(X_test)

print(y_predict)

accuracy = accuracy_score(y_test, y_predict)

print('\n',"Accuracy: ", accuracy) 

IGNORE

features = ["Department"]
X = df_train[features]
y = df_train['Stay']
X

Radiotherapy = 0
anesthesia = 1
gynecology = 2
TB & Chest Disease = 3


X.tail(20)